### make_lstm_classification_model_dataset
* input: 
    + ten seqence of movie_vector(word2vec embedding) 
    + view_duration(normalize) 
    + diff_before_view 
    + month 
    + day of month 
    + day of week


* ouput: 
    + after ten_veiw, movie_id

In [1]:
import sys
sys.path.append("../../sktnb_code_challenge/ttuckbaegi3/metrics")
import pandas as pd
%matplotlib inline

In [2]:
raw_df = pd.read_csv('../../ sample.csv')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15823019 entries, 0 to 15823018
Data columns (total 5 columns):
USER_ID      int64
MOVIE_ID     int64
DURATION     int64
WATCH_DAY    int64
WATCH_SEQ    int64
dtypes: int64(5)
memory usage: 603.6 MB


### movie id -> embedding vectors

In [3]:
from gensim.models import Word2Vec

In [7]:
model = Word2Vec.load('../../movie_w2v_mincount_model.model')

In [8]:
model.wv[['0','1']].shape

(2, 100)

[5026,
 3851,
 2608,
 5775,
 7322,
 1330,
 4130,
 824,
 7382,
 2420,
 5375,
 2340,
 28,
 6908,
 4820,
 1068,
 6428,
 8084,
 3956,
 3987,
 4129,
 5694,
 3549,
 5746,
 4078,
 5382,
 2051,
 1645,
 4009,
 1400,
 6411,
 2119,
 4959,
 7285,
 4426,
 2493,
 1371,
 7691,
 603,
 3212,
 4949,
 583,
 6561,
 884,
 2929,
 1720,
 5657,
 3612,
 7812,
 4248,
 4023,
 7824,
 1486,
 7870,
 1887,
 3262,
 6127,
 1448,
 620,
 406,
 6816,
 8032,
 4802,
 753,
 8158,
 2127,
 5726,
 3401,
 29,
 2492,
 5967,
 762,
 1984,
 5162,
 4262,
 5767,
 1841,
 1904,
 280,
 4388,
 5097,
 3933,
 8210,
 7607,
 641,
 3092,
 2181,
 2658,
 2770,
 7760,
 1844,
 4501,
 5905,
 5354,
 8046,
 7718,
 3139,
 845,
 5227,
 4606,
 401,
 5234,
 2621,
 6486,
 829,
 1845,
 2540,
 1092,
 4362,
 6885,
 6035,
 6849,
 3477,
 5249,
 4790,
 4094,
 704,
 7029,
 6189,
 7917,
 6549,
 3945,
 8075,
 4155,
 6516,
 1998,
 1986,
 4306,
 3167,
 1363,
 194,
 7238,
 4171,
 3618,
 411,
 5533,
 3754,
 508,
 2559,
 2204,
 3330,
 1167,
 8082,
 2537,
 1559,
 7035,

In [17]:
model.wv[['5026','5375']]

array([[ 0.44806391,  0.0953628 , -0.19928835, -0.12690076, -0.11037495,
        -0.35596514,  0.4106898 , -0.18596853, -0.34542349, -0.20176937,
        -0.2227519 , -0.41232592, -0.2382734 , -0.0698659 , -0.13645069,
        -0.12030501, -0.24848665,  0.30903777, -0.40315208,  0.22866532,
        -0.24976894,  0.13106757, -0.0924192 ,  0.11868199,  0.5807699 ,
         0.17382151,  0.07294977,  0.16433975, -0.04419158,  0.05384754,
        -0.21274933,  0.364723  , -0.05993083, -0.21902199, -0.43793106,
        -0.11133868,  0.01512   , -0.20699683,  0.16902085, -0.18678471,
         0.00305048, -0.08853789, -0.08498001,  0.26610357,  0.00195898,
         0.10567134,  0.26797348,  0.03540903,  0.25870806,  0.36149186,
        -0.21928415, -0.19035029, -0.05398019,  0.25878239, -0.09399482,
         0.00289247, -0.18100415,  0.10099125, -0.10762728,  0.3374131 ,
        -0.20881411,  0.22981411,  0.07032824,  0.24143688,  0.16974629,
        -0.08220121,  0.3460156 ,  0.24184902,  0.2

In [18]:
model.wv[raw_df['MOVIE_ID'].value_counts().index.tolist()]

KeyError: "word '5026' not in vocabulary"

### duration scale -1 to 1

In [4]:
from sklearn.preprocessing import minmax_scale

In [5]:
raw_df['scaled_duration'] = minmax_scale(raw_df.DURATION.values,feature_range=(-1,1))
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15823019 entries, 0 to 15823018
Data columns (total 6 columns):
USER_ID            int64
MOVIE_ID           int64
DURATION           int64
WATCH_DAY          int64
WATCH_SEQ          int64
scaled_duration    float64
dtypes: float64(1), int64(5)
memory usage: 724.3 MB


/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, _DataConversionWarning)


### date to 2 dim circle point

In [6]:
from datetime import datetime

In [7]:
num_values = 7

degrees_week = np.linspace(0,360,num_values + 1)[:-1]
print(degrees_week)

sin_week = np.sin(np.deg2rad(degrees_week))
print(sin_week)

cos_week = np.cos(np.deg2rad(degrees_week))
print(cos_week)

[   0.           51.42857143  102.85714286  154.28571429  205.71428571
  257.14285714  308.57142857]
[ 0.          0.78183148  0.97492791  0.43388374 -0.43388374 -0.97492791
 -0.78183148]
[ 1.          0.6234898  -0.22252093 -0.90096887 -0.90096887 -0.22252093
  0.6234898 ]


In [8]:
num_values = 12

degrees_month = np.linspace(0,360,num_values + 1)[:-1]
print(degrees_month)

sin_month = np.sin(np.deg2rad(degrees_month))
print(sin_month)

cos_month = np.cos(np.deg2rad(degrees_month))
print(cos_month)

[   0.   30.   60.   90.  120.  150.  180.  210.  240.  270.  300.  330.]
[  0.00000000e+00   5.00000000e-01   8.66025404e-01   1.00000000e+00
   8.66025404e-01   5.00000000e-01   1.22464680e-16  -5.00000000e-01
  -8.66025404e-01  -1.00000000e+00  -8.66025404e-01  -5.00000000e-01]
[  1.00000000e+00   8.66025404e-01   5.00000000e-01   6.12323400e-17
  -5.00000000e-01  -8.66025404e-01  -1.00000000e+00  -8.66025404e-01
  -5.00000000e-01  -1.83697020e-16   5.00000000e-01   8.66025404e-01]


In [9]:
num_values = 31

degrees_day = np.linspace(0,360,num_values + 1)[:-1]
print(degrees_day)

sin_day = np.sin(np.deg2rad(degrees_day))
print(sin_day)

cos_day = np.cos(np.deg2rad(degrees_day))
print(cos_day)

[   0.           11.61290323   23.22580645   34.83870968   46.4516129
   58.06451613   69.67741935   81.29032258   92.90322581  104.51612903
  116.12903226  127.74193548  139.35483871  150.96774194  162.58064516
  174.19354839  185.80645161  197.41935484  209.03225806  220.64516129
  232.25806452  243.87096774  255.48387097  267.09677419  278.70967742
  290.32258065  301.93548387  313.5483871   325.16129032  336.77419355
  348.38709677]
[ 0.          0.20129852  0.39435586  0.57126822  0.72479279  0.84864426
  0.93775213  0.98846832  0.99871651  0.96807712  0.89780454  0.79077574
  0.65137248  0.48530196  0.29936312  0.10116832 -0.10116832 -0.29936312
 -0.48530196 -0.65137248 -0.79077574 -0.89780454 -0.96807712 -0.99871651
 -0.98846832 -0.93775213 -0.84864426 -0.72479279 -0.57126822 -0.39435586
 -0.20129852]
[ 1.          0.97952994  0.91895781  0.82076344  0.68896692  0.52896401
  0.34730525  0.15142778 -0.05064917 -0.25065253 -0.44039415 -0.61210598
 -0.75875812 -0.87434662 -0.954139

In [10]:
def make_coord(date,num_class):
    # 1 ~ 12, 1 ~ 7, 1 ~ 31
    degrees = np.linspace(0,360,num_values + 1)[:-1]
    sin = np.sin(np.deg2rad(degrees))
    cos = np.cos(np.deg2rad(degrees))
    return sin, cos

In [11]:
def date2coord(date, sin, cos):
    return cos[date-1],sin[date-1]

In [12]:
int(str(raw_df.WATCH_DAY[0])[4:6])

8

In [13]:
int(str(raw_df.WATCH_DAY[0])[6:])

15

In [14]:
%%time
raw_df['watch_month'] = raw_df['WATCH_DAY'].apply(lambda x: int(str(x)[4:6]))
raw_df['watch_day'] = raw_df['WATCH_DAY'].apply(lambda x: int(str(x)[6:]))
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15823019 entries, 0 to 15823018
Data columns (total 8 columns):
USER_ID            int64
MOVIE_ID           int64
DURATION           int64
WATCH_DAY          int64
WATCH_SEQ          int64
scaled_duration    float64
watch_month        int64
watch_day          int64
dtypes: float64(1), int64(7)
memory usage: 965.8 MB
CPU times: user 29 s, sys: 1.69 s, total: 30.6 s
Wall time: 30.8 s


In [15]:
raw_df.watch_day.value_counts()

6     724432
2     634778
3     623128
5     582768
7     582512
9     578890
4     553427
16    548477
1     548361
11    545293
10    531010
23    520366
12    519603
20    506035
17    503696
8     495883
13    495205
24    494243
21    493745
30    486732
19    482074
26    480361
25    475486
18    472651
27    458207
14    456563
15    442898
28    437044
22    426944
29    397647
31    324560
Name: watch_day, dtype: int64

In [16]:
raw_df.watch_month.value_counts()

1     3712150
12    3202152
11    2485990
8     2283535
10    2195564
9     1943628
Name: watch_month, dtype: int64

In [17]:
def date2week(x):
    y = int(str(x)[:4])
    m = int(str(x)[4:6])
    d = int(str(x)[6:])
    day = datetime(y,m,d)
    return day.weekday()

In [18]:
raw_df['week'] = raw_df.WATCH_DAY.apply(date2week)
raw_df['week'] = raw_df['week'] + 1
raw_df['week'].value_counts()

6    3420047
7    2940547
1    1958997
5    1913684
3    1910585
2    1862975
4    1816184
Name: week, dtype: int64

In [19]:
%%time
raw_df['week_coord'] = raw_df['week'].apply(date2coord, args=(sin_week,cos_week,))

CPU times: user 12.2 s, sys: 583 ms, total: 12.8 s
Wall time: 12.8 s


In [20]:
raw_df['day_coord'] = raw_df['watch_day'].apply(date2coord, args=(sin_day,cos_day,))

In [21]:
raw_df['month_coord'] = raw_df['watch_month'].apply(date2coord, args=(sin_month,cos_month))

In [22]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15823019 entries, 0 to 15823018
Data columns (total 12 columns):
USER_ID            int64
MOVIE_ID           int64
DURATION           int64
WATCH_DAY          int64
WATCH_SEQ          int64
scaled_duration    float64
watch_month        int64
watch_day          int64
week               int64
week_coord         object
day_coord          object
month_coord        object
dtypes: float64(1), int64(8), object(3)
memory usage: 1.4+ GB


In [23]:
date_format = "%Y%m%d"
raw_df['datetime_watch_day'] = raw_df['WATCH_DAY'].apply(lambda x:datetime.strptime(str(x),date_format))

In [24]:
raw_df['diff_day'] = raw_df['datetime_watch_day'] - raw_df['datetime_watch_day'].shift(-1)

In [25]:
raw_df.dropna(inplace=True)

In [26]:
raw_df['diff_day'] = raw_df['diff_day'].apply(lambda x:x.days)

### too big, sampling 

In [28]:
sample_id = raw_df['USER_ID'].value_counts().index.tolist()[:40000]
#sample_id

In [30]:
raw_df.set_index('USER_ID',inplace=True)

In [32]:
sample = raw_df.loc[sample_id]
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3472936 entries, 97140 to 67965
Data columns (total 13 columns):
MOVIE_ID              int64
DURATION              int64
WATCH_DAY             int64
WATCH_SEQ             int64
scaled_duration       float64
watch_month           int64
watch_day             int64
week                  int64
week_coord            object
day_coord             object
month_coord           object
datetime_watch_day    datetime64[ns]
diff_day              int64
dtypes: datetime64[ns](1), float64(1), int64(8), object(3)
memory usage: 370.9+ MB


In [33]:
sample.reset_index(inplace=True)
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3472936 entries, 0 to 3472935
Data columns (total 14 columns):
USER_ID               int64
MOVIE_ID              int64
DURATION              int64
WATCH_DAY             int64
WATCH_SEQ             int64
scaled_duration       float64
watch_month           int64
watch_day             int64
week                  int64
week_coord            object
day_coord             object
month_coord           object
datetime_watch_day    datetime64[ns]
diff_day              int64
dtypes: datetime64[ns](1), float64(1), int64(9), object(3)
memory usage: 370.9+ MB


In [34]:
sample.head()

,USER_ID,MOVIE_ID,DURATION,WATCH_DAY,WATCH_SEQ,scaled_duration,watch_month,watch_day,week,week_coord,day_coord,month_coord,datetime_watch_day,diff_day
0,97140,958,29,20170801,1,-0.718593,8,1,2,"(0.623489801859, 0.781831482468)","(1.0, 0.0)","(-0.866025403784, -0.5)",2017-08-01,-2
1,97140,3987,70,20170803,1,-0.306533,8,3,4,"(-0.900968867902, 0.433883739118)","(0.91895781162, 0.394355855113)","(-0.866025403784, -0.5)",2017-08-03,-1
2,97140,4949,24,20170804,1,-0.768844,8,4,5,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",2017-08-04,-8
3,97140,1068,3,20170812,1,-0.979899,8,12,6,"(-0.222520933956, -0.974927912182)","(-0.612105982548, 0.790775736938)","(-0.866025403784, -0.5)",2017-08-12,0
4,97140,1068,28,20170812,2,-0.728643,8,12,6,"(-0.222520933956, -0.974927912182)","(-0.612105982548, 0.790775736938)","(-0.866025403784, -0.5)",2017-08-12,0


### ten sequence and diff day

In [35]:
raw_df = sample

In [36]:
import time

In [37]:
%%time
result = []
for i, index in enumerate(raw_df['USER_ID'].value_counts().index):
    st = time.time()
    # 한유저의 마지막 diff_day 는 다른유저와의 차이임으로 제거
    sub = raw_df[raw_df['USER_ID'] == index].sort_values(['WATCH_DAY', 'WATCH_SEQ']).head(-1)

    # make train, label
    inseq = 10
    names = []
    temp_list = []
    features = ['MOVIE_ID','scaled_duration','diff_day','week_coord','day_coord','month_coord']
    
    for k in range(inseq):
        names += [(feature_name+'(t-%d)' % k) for feature_name in features]
        temp_list.append(sub[features].shift(-1*k))
        if k == inseq - 1:
            names += [('MOVIE_ID'+'(t-%d)' % inseq)]
            temp_list.append(sub['MOVIE_ID'].shift(-1*inseq))
            
    df = pd.concat(temp_list, axis=1)
    df.columns = names
    df.dropna(inplace=True)
    result.append(df)
    et = time.time()
    
    if i % 1000 == 0:
        print(i, index, (et-st)*1000)
        
#     if i == 10 :
#         break

0 285658 34.993886947631836
1000 101296 16.555070877075195
2000 143991 21.939992904663086
3000 79300 17.938852310180664
4000 240986 17.37189292907715
5000 350976 17.73214340209961
6000 146328 16.71290397644043
7000 130820 20.355939865112305
8000 363012 17.681360244750977
9000 311875 21.235227584838867
10000 322815 16.566753387451172
11000 56019 21.086931228637695
12000 258314 15.925884246826172
13000 104246 16.153812408447266
14000 207883 20.04218101501465
15000 106294 15.733957290649414
16000 150284 16.47496223449707
17000 57556 16.894102096557617
18000 297406 17.41814613342285
19000 362704 15.635967254638672
20000 101769 17.897844314575195
21000 124548 20.19190788269043
22000 387511 16.620159149169922
23000 45249 16.67308807373047
24000 109549 19.256114959716797
25000 18998 15.789985656738281
26000 78550 17.182111740112305
27000 152111 16.901016235351562
28000 178464 16.351938247680664
29000 387933 16.05820655822754
30000 299357 16.77417755126953
31000 170739 16.55125617980957
32000 

### feature join

In [38]:
len(result)

40000

In [43]:
%%time
sample = pd.concat(result,axis=0)

CPU times: user 1min 44s, sys: 1.82 s, total: 1min 46s
Wall time: 1min 46s


In [44]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3032936 entries, 15700 to 3461674
Data columns (total 61 columns):
MOVIE_ID(t-0)           int64
scaled_duration(t-0)    float64
diff_day(t-0)           int64
week_coord(t-0)         object
day_coord(t-0)          object
month_coord(t-0)        object
MOVIE_ID(t-1)           float64
scaled_duration(t-1)    float64
diff_day(t-1)           float64
week_coord(t-1)         object
day_coord(t-1)          object
month_coord(t-1)        object
MOVIE_ID(t-2)           float64
scaled_duration(t-2)    float64
diff_day(t-2)           float64
week_coord(t-2)         object
day_coord(t-2)          object
month_coord(t-2)        object
MOVIE_ID(t-3)           float64
scaled_duration(t-3)    float64
diff_day(t-3)           float64
week_coord(t-3)         object
day_coord(t-3)          object
month_coord(t-3)        object
MOVIE_ID(t-4)           float64
scaled_duration(t-4)    float64
diff_day(t-4)           float64
week_coord(t-4)         object
day_

In [45]:
sample.head()

,MOVIE_ID(t-0),scaled_duration(t-0),diff_day(t-0),week_coord(t-0),day_coord(t-0),month_coord(t-0),MOVIE_ID(t-1),scaled_duration(t-1),diff_day(t-1),week_coord(t-1),...,week_coord(t-8),day_coord(t-8),month_coord(t-8),MOVIE_ID(t-9),scaled_duration(t-9),diff_day(t-9),week_coord(t-9),day_coord(t-9),month_coord(t-9),MOVIE_ID(t-10)
15700,5657,-0.909548,0,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",5657.0,-0.929648,0.0,"(-0.900968867902, -0.433883739118)",...,"(0.623489801859, -0.781831482468)","(0.528964010327, 0.848644257495)","(-0.866025403784, -0.5)",2540.0,-0.989950,-10.0,"(-0.222520933956, 0.974927912182)","(-0.0506491688387, 0.998716507171)","(-0.866025403784, -0.5)",5375.0
15701,5657,-0.929648,0,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",5657.0,-0.839196,-1.0,"(-0.900968867902, -0.433883739118)",...,"(-0.222520933956, 0.974927912182)","(-0.0506491688387, 0.998716507171)","(-0.866025403784, -0.5)",5375.0,-0.929648,-12.0,"(-0.222520933956, -0.974927912182)","(-0.874346616145, -0.485301962531)","(-0.866025403784, -0.5)",5375.0
15702,5657,-0.839196,-1,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",5657.0,-0.165829,-1.0,"(-0.222520933956, -0.974927912182)",...,"(-0.222520933956, -0.974927912182)","(-0.874346616145, -0.485301962531)","(-0.866025403784, -0.5)",5375.0,-0.809045,-4.0,"(-0.900968867902, 0.433883739118)","(0.979529941252, -0.201298520089)","(-0.866025403784, -0.5)",3549.0
15703,5657,-0.165829,-1,"(-0.222520933956, -0.974927912182)","(0.688966919076, 0.724792787229)","(-0.866025403784, -0.5)",406.0,-0.638191,0.0,"(0.623489801859, -0.781831482468)",...,"(-0.900968867902, 0.433883739118)","(0.979529941252, -0.201298520089)","(-0.866025403784, -0.5)",3549.0,-1.000000,0.0,"(1.0, 0.0)","(0.820763441207, 0.571268215095)","(-0.5, -0.866025403784)",3549.0
15704,406,-0.638191,0,"(0.623489801859, -0.781831482468)","(0.528964010327, 0.848644257495)","(-0.866025403784, -0.5)",406.0,-0.909548,0.0,"(0.623489801859, -0.781831482468)",...,"(1.0, 0.0)","(0.820763441207, 0.571268215095)","(-0.5, -0.866025403784)",3549.0,-0.849246,0.0,"(1.0, 0.0)","(0.820763441207, 0.571268215095)","(-0.5, -0.866025403784)",3549.0


In [46]:
del raw_df

In [47]:
sample.reset_index(drop=True,inplace=True)

In [48]:
sample.head()

,MOVIE_ID(t-0),scaled_duration(t-0),diff_day(t-0),week_coord(t-0),day_coord(t-0),month_coord(t-0),MOVIE_ID(t-1),scaled_duration(t-1),diff_day(t-1),week_coord(t-1),...,week_coord(t-8),day_coord(t-8),month_coord(t-8),MOVIE_ID(t-9),scaled_duration(t-9),diff_day(t-9),week_coord(t-9),day_coord(t-9),month_coord(t-9),MOVIE_ID(t-10)
0,5657,-0.909548,0,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",5657.0,-0.929648,0.0,"(-0.900968867902, -0.433883739118)",...,"(0.623489801859, -0.781831482468)","(0.528964010327, 0.848644257495)","(-0.866025403784, -0.5)",2540.0,-0.989950,-10.0,"(-0.222520933956, 0.974927912182)","(-0.0506491688387, 0.998716507171)","(-0.866025403784, -0.5)",5375.0
1,5657,-0.929648,0,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",5657.0,-0.839196,-1.0,"(-0.900968867902, -0.433883739118)",...,"(-0.222520933956, 0.974927912182)","(-0.0506491688387, 0.998716507171)","(-0.866025403784, -0.5)",5375.0,-0.929648,-12.0,"(-0.222520933956, -0.974927912182)","(-0.874346616145, -0.485301962531)","(-0.866025403784, -0.5)",5375.0
2,5657,-0.839196,-1,"(-0.900968867902, -0.433883739118)","(0.820763441207, 0.571268215095)","(-0.866025403784, -0.5)",5657.0,-0.165829,-1.0,"(-0.222520933956, -0.974927912182)",...,"(-0.222520933956, -0.974927912182)","(-0.874346616145, -0.485301962531)","(-0.866025403784, -0.5)",5375.0,-0.809045,-4.0,"(-0.900968867902, 0.433883739118)","(0.979529941252, -0.201298520089)","(-0.866025403784, -0.5)",3549.0
3,5657,-0.165829,-1,"(-0.222520933956, -0.974927912182)","(0.688966919076, 0.724792787229)","(-0.866025403784, -0.5)",406.0,-0.638191,0.0,"(0.623489801859, -0.781831482468)",...,"(-0.900968867902, 0.433883739118)","(0.979529941252, -0.201298520089)","(-0.866025403784, -0.5)",3549.0,-1.000000,0.0,"(1.0, 0.0)","(0.820763441207, 0.571268215095)","(-0.5, -0.866025403784)",3549.0
4,406,-0.638191,0,"(0.623489801859, -0.781831482468)","(0.528964010327, 0.848644257495)","(-0.866025403784, -0.5)",406.0,-0.909548,0.0,"(0.623489801859, -0.781831482468)",...,"(1.0, 0.0)","(0.820763441207, 0.571268215095)","(-0.5, -0.866025403784)",3549.0,-0.849246,0.0,"(1.0, 0.0)","(0.820763441207, 0.571268215095)","(-0.5, -0.866025403784)",3549.0


In [ ]:
sample.to_csv('../../lstm_data_sample.csv')